### **Keyword clustering**
- One-hot embeddings (https://colab.research.google.com/drive/1HHNFjKlip1AaFIuvvn0AicWyv6egLOZw?usp=sharing#scrollTo=aNZQMs7xZzgv)
- Sentence transformers 
- word2vec

In [1]:
import pandas as pd

*One-hot embedding*

In [2]:
base_path = '../05-transformation/'
acteur = 'chum'
file_path = base_path + acteur + '_weighting_OKapiBM25.csv'
with open(file_path, encoding='utf-8') as f:
    keywords = pd.read_csv(f)[['Terme', 'Fréquence (TF)', 'Fréquence documentaire (DF)']]

keywords['TF + DF'] = keywords['Fréquence (TF)'] + keywords['Fréquence documentaire (DF)']

keywords = keywords.rename(columns={'Terme': 'Keyword'})
keywords

,Keyword,Fréquence (TF),Fréquence documentaire (DF),TF + DF
0,chirurgiens du canada,46,46,92
1,réunions hebdomadaires,86,86,172
2,centre hospitalier de l'université,115,88,203
3,activité de développement professionnel,43,43,86
4,centre de recherche du centre,78,68,146
...,...,...,...,...
179,professeur au département,34,26,60
180,chercheurs du crchum,37,30,67
181,recherche chirurgie,33,32,65
182,calendrier des conférences,40,40,80


In [3]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer("(\w+\'|\w+-\w+|\(|\)|\w+)")

file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stop = [t.lower().strip('\n') for t in f.readlines()]

def to_tokens(kw, min_chars=2):
    tokens = tokenizer.tokenize(str(kw)) # split the string into a list of words
    tokens = [word for word in tokens if len(word) > min_chars] 
    tokens = [str(word) for word in tokens if word not in stop] 
    
    tokens = set(tokens) # to remove duplicates
    tokens = sorted(tokens) # converts our set back to a list and sorts words in alphabetical order
    return tokens


In [4]:
keywords["tokens"] = keywords["Keyword"].apply(lambda x: to_tokens(
    x,
    min_chars=3,
))
keywords

,Keyword,Fréquence (TF),Fréquence documentaire (DF),TF + DF,tokens
0,chirurgiens du canada,46,46,92,"[canada, chirurgiens]"
1,réunions hebdomadaires,86,86,172,"[hebdomadaires, réunions]"
2,centre hospitalier de l'université,115,88,203,"[centre, hospitalier, université]"
3,activité de développement professionnel,43,43,86,"[activité, développement, professionnel]"
4,centre de recherche du centre,78,68,146,"[centre, recherche]"
...,...,...,...,...,...
179,professeur au département,34,26,60,"[département, professeur]"
180,chercheurs du crchum,37,30,67,"[chercheurs, crchum]"
181,recherche chirurgie,33,32,65,"[chirurgie, recherche]"
182,calendrier des conférences,40,40,80,"[calendrier, conférences]"


In [5]:
vocab = sorted(set(keywords["tokens"].explode()))
len(vocab)

156

In [6]:
def to_vector(keyword,vocab):
    """
    Calculates vector of keyword on given vocabulary.

    Returns vector as a list of values.  
    """
    vector = []
    for word in vocab:
        if word in keyword:
            vector.append(1)
        else:
            vector.append(0)
    return vector

keywords["vector"] = keywords["tokens"].apply(lambda x: to_vector(x,vocab))
keywords.head()

,Keyword,Fréquence (TF),Fréquence documentaire (DF),TF + DF,tokens,vector
0,chirurgiens du canada,46,46,92,"[canada, chirurgiens]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,réunions hebdomadaires,86,86,172,"[hebdomadaires, réunions]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,centre hospitalier de l'université,115,88,203,"[centre, hospitalier, université]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,activité de développement professionnel,43,43,86,"[activité, développement, professionnel]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,centre de recherche du centre,78,68,146,"[centre, recherche]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


*Sentence transformers embedding*

---------------------------------------------------------------------------

In [7]:
from sentence_transformers import SentenceTransformer, models
import torch

# On va utiliser un modèle BERT/sentence transformers (fr) pour extraire nos embeddings plutôt que des simples one-hot encoding
model =  SentenceTransformer("dangvantuan/sentence-camembert-base")

sentences = keywords['Keyword'].tolist()
embeddings_st = model.encode(sentences, convert_to_numpy=True)

c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:560: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


### *KMeans*

*One-Hot embeddings*

In [8]:
from sklearn.cluster import KMeans

X = keywords["vector"].to_list()

### **Déterminer K**   
**Score "intersection"** *(lower is better)*  
Pour chaque cluster, on cherche s'il existe une intersection entre les termes qui le constituent ; pour chaque cluster pour lequel il n'existe pas d'intersection,
on ajoute 1 au score ; au final, on va retenir la plus petite valeur de k pour laquelle le score = 0 (si elle existe) 

**Score "orphelins"** *(lower is better)*  
Pour chaque cluster qui ne contient qu'un seul terme (*ie* orphelin), on ajoute 1 au score ; au final, on va retenir la plus petite valeur de k pour laquelle le score = 0 (si elle existe)  

-----------------------------------------------------------------------------------

*Pour combiner les deux scores*  
On pourrait faire la somme du score "intersection" et du score "orphelin" et retenir la valeur k pour laquelle ce score est la plus faible ; ou encore, la valeur k la plus faible pour laquelle les deux scores = 0 (si elle existe).  

On pourrait aussi privilégier un des deux scores, retenir d'abord la plus petite valeur de k pour laquelle le score = 0, puis ensuite retenir la valeur minimale associée à l'autre score.

In [9]:
rg = range(round(len(X)/5), len(set(tuple(x) for x in X)))
scores = [{'algorithme' : 'K-means', 'embedding' : 'One-Hot', 'k' : x, 'score_intersection' : None, 'score_orphelin': None} for x in rg]

rg

range(37, 174)

In [10]:
for k in range(len(scores)):
    scores[k]['score_intersection'] = 0
    scores[k]['score_orphelin'] = 0

    kmeans = KMeans(n_clusters = scores[k]['k'], init='k-means++', algorithm='elkan', random_state=0, n_init=1, max_iter=200).fit(X)
    keywords["kmeans"] = list(kmeans.labels_)

    labels = set(kmeans.labels_.tolist())
    for label in labels:
        d = keywords[keywords['kmeans'] == label]['tokens'].tolist()
        new_label = list(set.intersection(*map(set,d)))

        # Si on ne trouve pas d'intersection entre les termes d'un même cluster, on ajoute 1 au score intersection ; 
        # au final, on va retenir la plus petite valeur de k pour laquelle le score = 0 (si elle existe) 
        if(len(new_label) == 0):
            scores[k]['score_intersection'] += 1

        ## Si le cluster ne contient qu'un seul terme, on ajoute 1 au score orphelin
        if(len(d) == 1):
            scores[k]['score_orphelin'] += 1

tab_scores = pd.DataFrame.from_records(scores)
tab_scores['Score'] = tab_scores['score_intersection'] + tab_scores['score_orphelin']

tab_scores.sort_values(['score_intersection'])

try:
    # On retient un sous-ensemble de valeurs pour lesquelles le score intersection est nul (on sait qu'il y a toujours une intersection entre nos clusters)
    candidats =  tab_scores[tab_scores['score_intersection'] == 0]

    # Parmis ces valeurs, on retient celle pour laquelle le score_orphelin est le plus bas
    k = candidats[candidats['score_orphelin'] == candidats['score_orphelin'].min()]['k'].values[0]

except Exception as e:
    k = tab_scores[tab_scores['Score'] == tab_scores['Score'].min()]['k'].values[0]
    print("NOT OK")

print("K = " + str(k))

kmeans = KMeans(n_clusters = k, init='k-means++', algorithm='elkan', random_state=0, n_init=1, max_iter=200).fit(X)
keywords["kmeans"] = list(kmeans.labels_)

labels = set(kmeans.labels_.tolist())
desired_labels = {x : None for x in labels} # (on initialise à None)
for label in labels:
    d = keywords[keywords['kmeans'] == label]['tokens'].tolist()
    new_label = list(set.intersection(*map(set,d)))

    try:
        desired_labels[label] = new_label[0]

    except Exception as e:
        cluster = keywords[keywords["kmeans"] == label]
        max_freq = cluster['TF + DF'].max()
        new_label = cluster[cluster['TF + DF'] == max_freq]['Keyword'].values
        desired_labels[label] = new_label[0]

keywords['Cluster'] = keywords['kmeans'].map(desired_labels)

keywords.sort_values(["Cluster"], 
        axis=0,
        ascending=[False], 
        inplace=True)

keywords[['Cluster', 'Keyword', 'Fréquence (TF)']].sort_values('Cluster').to_csv('../06-clustering/param_finaux_08-18-2022.csv')

K = 86


*Sentence transformers embeddings*

In [11]:
X = embeddings_st

In [12]:
for k in range(len(scores)):
    scores[k]['score_intersection'] = 0
    scores[k]['score_orphelin'] = 0

    kmeans = KMeans(n_clusters = scores[k]['k'], init='k-means++', algorithm='elkan', random_state=0, n_init=1, max_iter=200).fit(X)
    keywords["kmeans"] = list(kmeans.labels_)

    labels = set(kmeans.labels_.tolist())
    for label in labels:
        d = keywords[keywords['kmeans'] == label]['tokens'].tolist()
        new_label = list(set.intersection(*map(set,d)))

        # Si on ne trouve pas d'intersection entre les termes d'un même cluster, on ajoute 1 au score intersection ; 
        # au final, on va retenir la plus petite valeur de k pour laquelle le score = 0 (si elle existe) 
        if(len(new_label) == 0):
            scores[k]['score_intersection'] += 1

        ## Si le cluster ne contient qu'un seul terme, on ajoute 1 au score orphelin
        if(len(d) == 1):
            scores[k]['score_orphelin'] += 1

tab_scores = pd.DataFrame.from_records(scores)
tab_scores['Score'] = tab_scores['score_intersection'] + tab_scores['score_orphelin']

tab_scores.sort_values(['score_intersection'])

try:
    # On retient un sous-ensemble de valeurs pour lesquelles le score intersection est nul (on sait qu'il y a toujours une intersection entre nos clusters)
    candidats =  tab_scores[tab_scores['score_intersection'] == 0]

    # Parmis ces valeurs, on retient celle pour laquelle le score_orphelin est le plus bas
    k = candidats[candidats['score_orphelin'] == candidats['score_orphelin'].min()]['k'].values[0]

except Exception as e:
    print("NOT OK")
    k = tab_scores[tab_scores['Score'] == tab_scores['Score'].min()]['k'].values[0]

print("K = " + str(k))

kmeans = KMeans(n_clusters = k, init='k-means++', algorithm='elkan', random_state=0, n_init=1, max_iter=200).fit(X)
keywords["kmeans"] = list(kmeans.labels_)

labels = set(kmeans.labels_.tolist())
desired_labels = {x : None for x in labels} # (on initialise à None)
for label in labels:
    d = keywords[keywords['kmeans'] == label]['tokens'].tolist()
    new_label = list(set.intersection(*map(set,d)))

    try:
        desired_labels[label] = new_label[0]

    except Exception as e:
        cluster = keywords[keywords["kmeans"] == label]
        max_freq = cluster['TF + DF'].max()
        new_label = cluster[cluster['TF + DF'] == max_freq]['Keyword'].values
        desired_labels[label] = new_label[0]

keywords['Cluster'] = keywords['kmeans'].map(desired_labels)

keywords.sort_values(["Cluster"], 
        axis=0,
        ascending=[False], 
        inplace=True)

keywords[['Cluster', 'Keyword', 'Fréquence (TF)']].sort_values('Cluster')

NOT OK
K = 37


,Cluster,Keyword,Fréquence (TF)
16,assistance immédiate,département de radiologie,101
153,assistance immédiate,immunopathologie professeur,32
180,assistance immédiate,chercheurs du crchum,37
73,assistance immédiate,axe de recherche neurosciences,107
38,assistance immédiate,assistance immédiate,912
...,...,...,...
26,researchgate recherche,travaux de recherche,49
63,researchgate recherche,researchgate recherche,168
11,titulaire de la chaire,titulaire de la chaire,48
60,titulaire de la chaire,département de psychiatrie,39


### *Expectation-Maximization*

*One-Hot embedding*

In [13]:
from sklearn.mixture import GaussianMixture

X = keywords["vector"].to_list()

In [14]:
rg = range(round(len(X)/5), len(set(tuple(x) for x in X)))
scores = [{'algorithme' : 'Expectation-Maximization', 'embedding' : 'One-Hot', 'k' : x, 'score_intersection' : None, 'score_orphelin': None} for x in rg]

rg

range(37, 174)

In [15]:
for k in range(len(scores)):
    scores[k]['score_intersection'] = 0
    scores[k]['score_orphelin'] = 0

    gmm = GaussianMixture(n_components=scores[k]['k'], init_params='k-means++', covariance_type='diag').fit(X) # diag pour gérer MemoryError
    keywords["gmm"] = list(gmm.predict(X))

    labels = gmm.predict(X)

    labels = set(labels)
    for label in labels:
        d = keywords[keywords['gmm'] == label]['tokens'].tolist()
        new_label = list(set.intersection(*map(set,d)))

        # Si on ne trouve pas d'intersection entre les termes d'un même cluster, on ajoute 1 au score intersection ; 
        # au final, on va retenir la plus petite valeur de k pour laquelle le score = 0 (si elle existe) 
        if(len(new_label) == 0):
            scores[k]['score_intersection'] += 1

        ## Si le cluster ne contient qu'un seul terme, on ajoute 1 au score orphelin
        if(len(d) == 1):
            scores[k]['score_orphelin'] += 1

tab_scores = pd.DataFrame.from_records(scores)
tab_scores['Score'] = tab_scores['score_intersection'] + tab_scores['score_orphelin']

tab_scores.sort_values(['score_intersection'])

try:
    # On retient un sous-ensemble de valeurs pour lesquelles le score intersection est nul (on sait qu'il y a toujours une intersection entre nos clusters)
    candidats =  tab_scores[tab_scores['score_intersection'] == 0]

    # Parmis ces valeurs, on retient celle pour laquelle le score_orphelin est le plus bas
    k = candidats[candidats['score_orphelin'] == candidats['score_orphelin'].min()]['k'].values[0]

except Exception as e:
    k = tab_scores[tab_scores['Score'] == tab_scores['Score'].min()]['k'].values[0]
    print("NOT OK")

print("K = " + str(k))


gmm = GaussianMixture(n_components=k, init_params='k-means++', covariance_type='diag').fit(X) # diag pour gérer MemoryError
keywords["gmm"] = list(gmm.predict(X))

labels = set(list(gmm.predict(X)))
desired_labels = {x : None for x in labels} # (on initialise à None)
for label in labels:
    d = keywords[keywords['gmm'] == label]['tokens'].tolist()
    new_label = list(set.intersection(*map(set,d)))

    try:
        desired_labels[label] = new_label[0]

    except Exception as e:
        cluster = keywords[keywords["gmm"] == label]
        max_freq = cluster['TF + DF'].max()
        new_label = cluster[cluster['TF + DF'] == max_freq]['Keyword'].values
        desired_labels[label] = new_label[0]

keywords['Cluster'] = keywords['gmm'].map(desired_labels)

keywords.sort_values(["Cluster"], 
        axis=0,
        ascending=[False], 
        inplace=True)

keywords[['Cluster', 'Keyword', 'Fréquence (TF)']].sort_values('Cluster')

K = 92


,Cluster,Keyword,Fréquence (TF)
6,adjointe,professeure adjointe,132
68,agrégée,professeure agrégée,62
24,animaux,modèles animaux,44
82,anne-marie,anne-marie mes-masson,43
88,assistance,besoin d'assistance immédiate,912
...,...,...,...
113,titulaire,chum professeur titulaire,44
108,titulaire,professeur titulaire,166
98,type,diabète de type,36
90,université,université de montréal titulaire,69
